# AutoGraph

tf.function 的一个很酷的新功能是 AutoGraph，它允许使用自然的 Python 语法编写图形代码

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

## tf.function 装饰器

当使用 tf.function 注释函数时，可以像调用任何其他函数一样调用它。 它将被编译成图，这意味着可以获得更快执行，更好地在 GPU 或 TPU 上运行或导出到 SavedModel。

In [2]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.79224634, 1.2441946 , 1.4953172 ],
       [0.56591576, 0.97079694, 1.1965573 ],
       [0.68131983, 1.2662568 , 1.540731  ]], dtype=float32)>

如果我们检查注释的结果，我们可以看到它是一个特殊的可调用函数，它处理与TensorFlow运行时的所有交互

In [3]:
simple_nn_layer

如果代码使用多个函数，则无需对它们进行全部注释,
**从带注释函数调用的任何函数也将以图形模式运行**

In [4]:
def linear_layer(x):
    return 2 * x + 1


@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: id=35, shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

## 使用 Python 控制流程

在 tf.function 中使用依赖于数据的控制流时，可以使用 Python 控制流语句，AutoGraph 会将它们转换为适当的 TensorFlow 操作。 例如，如果语句依赖于 Tensor，则语句将转换为 tf.cond()

In [5]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


AutoGraph 支持常见的 Python 语句，例如 while，if，break，continue 和 return，支持嵌套。 这意味着可以在 while 和 if 语句的条件下使用 Tensor 表达式，或者在 for 循环中迭代 Tensor

In [6]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=142, shape=(), dtype=int32, numpy=42>

AutoGraph 还为高级用户提供了低级 API。 例如，我们可以使用它来查看生成的代码

In [7]:
print(tf.autograph.to_code(sum_even.python_function, experimental_optional_features=None))

def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'sum_even_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sum_even_scope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
 

一个更复杂的控制流程的例子

In [8]:
@tf.function
def fizzbuzz(n):
    msg = tf.constant('')
    for i in tf.range(n):
        if tf.equal(i % 3, 0):
            msg += 'Fizz'
        elif tf.equal(i % 5, 0):
            msg += 'Buzz'
        else:
            msg += tf.as_string(i)
        msg += '\n'
    return msg

print(fizzbuzz(tf.constant(15)).numpy().decode())

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14



## Keras 和 AutoGraph

也可以将 tf.function 与对象方法一起使用。 例如，可以通过注释模型的调用函数来装饰自定义 Keras 模型

In [9]:
class CustomModel(tf.keras.models.Model):

    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data // 2

model = CustomModel()
model(tf.constant([-2, -4]))

<tf.Tensor: id=263, shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

**副作用**：就像在 eager 模式下一样，你可以使用带有副作用的操作，比如通常在 tf.function 中的 tf.assign 或 tf.print，它会插入必要的控件依赖项以确保它们按顺序执行

In [10]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v + 1)
    if tf.equal(v % 2, 0):
        v.assign(v + 1)


find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

## 用 AutoGraph 训练一个简单模型

In [11]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

# train_dataset = mnist_dataset()

model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(y, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    compute_accuracy(y, logits)
    return loss


@tf.function
def train(model, optimizer):
    train_ds = mnist_dataset()
    step = 0
    loss = 0.0
    accuracy = 0.0
    for x, y in train_ds:
        step += 1
        loss = train_one_step(model, optimizer, x, y)
        if tf.equal(step % 10, 0):
            tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
    return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.7814002 ; accuracy 0.304
Step 20 : loss 1.22172511 ; accuracy 0.4895
Step 30 : loss 0.801277697 ; accuracy 0.59
Step 40 : loss 0.777820647 ; accuracy 0.64575
Step 50 : loss 0.537724 ; accuracy 0.6908
Step 60 : loss 0.563764334 ; accuracy 0.722166657
Step 70 : loss 0.382106632 ; accuracy 0.746714294
Step 80 : loss 0.502317071 ; accuracy 0.7625
Step 90 : loss 0.476850361 ; accuracy 0.776111126
Step 100 : loss 0.318636358 ; accuracy 0.7881
Step 110 : loss 0.261523962 ; accuracy 0.799363613
Step 120 : loss 0.394204259 ; accuracy 0.807916641
Step 130 : loss 0.286721498 ; accuracy 0.815846145
Step 140 : loss 0.386131197 ; accuracy 0.822
Step 150 : loss 0.307116717 ; accuracy 0.8274
Step 160 : loss 0.317753911 ; accuracy 0.833437502
Step 170 : loss 0.289032489 ; accuracy 0.838470578
Step 180 : loss 0.435914338 ; accuracy 0.8435
Step 190 : loss 0.199862346 ; accuracy 0.848105252
Step 200 : loss 0.309946358 ; accuracy 0.85165
Final step tf.Tensor(200, shape=(), dtype=int32) : l

## 关于批处理的说明

在实际应用中，批处理对性能至关重要。 转换为 AutoGraph 的最佳代码是在批处理级别决定控制流的代码。 如果在单个示例级别做出决策，请尝试使用批处理 API 来维护性能

In [12]:
def square_if_positive(x):
    return [i ** 2 if i > 0 else i for i in x]

square_if_positive(range(-5, 5))

[-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

In [13]:
# 在tensorflow中上面的代码应该改成下面所示
@tf.function
def square_if_positive_naive(x):
    result = tf.TensorArray(tf.int32, size=x.shape[0])
    for i in tf.range(x.shape[0]):
        if x[i] > 0:
            result = result.write(i, x[i] ** 2)
        else:
            result = result.write(i, x[i])
    return result.stack()

square_if_positive_naive(tf.range(-5, 5))

<tf.Tensor: id=1186, shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>

In [14]:
# 推荐这么写 tf.where
def square_if_positive_vectorized(x):
    return tf.where(x > 0, x ** 2, x)

square_if_positive_vectorized(tf.range(-5, 5))

<tf.Tensor: id=1195, shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>